In [3]:
import youtube_dl
import sys
from requests import get

# URLs = [
#     'https://tv.naver.com/v/36179013', #test
#     'https://tv.naver.com/v/36072694', #source
#     'https://tv.naver.com/v/36073744', #highlight
# ]

class Downloader:
    def __init__(self, resolution):
        self.resolution = resolution
    
    def get_url(self ,url):
        result = ""
        ydl_opts = {
            # 'format': 'bestvideo/best', # best resolution?
        }
        ydl = youtube_dl.YoutubeDL(ydl_opts)
        info_dict = ydl.extract_info(url, download=False)
        print(info_dict)
        formats = info_dict.get('formats',None)
        for f in formats:
            if f.get('format_id', None) == 'avc1_' + self.resolution + 'P':
                result = f.get('url', None)
        print("\n\nresult >> ", result)
        return result
    
    def download(self, url, file_name):
        dl_url = self.get_url(url)
        with open(file_name + '.mp4', "wb") as file:   
            response = get(dl_url)
            print(response)
            file.write(response.content)

In [5]:
resolution = '720'
dl = Downloader(resolution)
file_name = "video_data.txt"

with open(file_name, "r") as f:
    for line in f:
        url, video_name = line.strip().split(",")
        dl.download(url, video_name)

[Naver] 28351574: Downloading JSON metadata
[Naver] 28351574: Downloading JSON metadata
{'id': '28351574', 'title': 'KT vs NC 풀영상 (2022.08.03)', 'formats': [{'format_id': 'HLS_144P', 'url': 'http://a01-g-cdn.vod.naver.net/navertv/c/read/v2/VOD_ALPHA/navertv_2022_08_03_45/hls/84942cb1-132e-11ed-9cb6-a0369ffdaa44.m3u8?__gda__=1685533129_4d841b6e4c4f253cea4b477a201ace8b', 'width': 256, 'height': 144, 'vbr': 82, 'abr': 64, 'filesize': None, 'protocol': 'm3u8_native', 'tbr': 146, 'ext': 'm3u8', 'format': 'HLS_144P - 256x144', 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3746.1 Safari/537.36', 'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.7', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Encoding': 'gzip, deflate', 'Accept-Language': 'en-us,en;q=0.5'}}, {'format_id': 'avc1_144P', 'url': 'http://a01-g-cdn.vod.naver.net/navertv/c/read/v2/VOD_ALPHA/navertv_2022_08_03_1557/dis

<Response [200]>


In [22]:
import cv2
import sys

class Video_capture:
    def __init__(self, video_path):
        self.cap = cv2.VideoCapture(video_path)
        self.ret = None 
        self.frame = None 
        
    def move_frame(self, frame_cnt):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_cnt)
        self.read()
        
    def make_grayscale(self):
        return cv2.cvtColor(self.frame, cv2.COLOR_BGR2GRAY)
    
    def read(self):
        self.ret, self.frame = self.cap.read()
        return

class Highlight_finder:
    def __init__(self, source, highlight, precision):
        self.name = source[:-4]
        self.source = Video_capture(source)
        self.highlight= Video_capture(highlight)
        self.frame_cnt = 0
        self.highlight_frame_cnt = 210 # 네이버 로고 지난 프레임 (210)
        self.precision = precision
        self.result = []
    
    def load_video(self):
        if not self.source.cap.isOpened():
                print('source video not opened')
                exit(-1)
        if not self.highlight.cap.isOpened():
                print('highlight video not opened')
                exit(-1)
        self.source.move_frame(self.frame_cnt)
        self.highlight.move_frame(self.highlight_frame_cnt)       
        return
    
    def match_template(self, src, frame):
        R = cv2.matchTemplate(src, frame, cv2.TM_CCORR_NORMED)
        _, maxVal, _, maxLoc = cv2.minMaxLoc(R)
        return maxVal
    
    def find_highlight(self):
        start = 0
        end = 0
        f = open(self.name +".txt",'w')
        while True: # 영상이 끝날때까지
            self.frame_cnt += 5
            if self.frame_cnt % 1800 == 0: # 1분마다 경과 체크
                print(self.frame_cnt / 1800, "min", self.highlight_frame_cnt)
            for i in range(5):
                self.source.read()
            if not self.source.ret or not self.highlight.ret:
                break
        #---------------------------------------------------------------------
            maxVal = self.match_template(self.source.make_grayscale(), self.highlight.make_grayscale())

            #cv2.imshow('source', self.source.frame)
            key = cv2.waitKey(25)
            if key == 27:
                cv2.destroyAllWindows()
                break

            if maxVal < self.precision: # 영상이 일치하는지 체크
                continue
            else: # 일치한다면
                result = maxVal
                if(self.frame_cnt - end > 150):
                    if end != 0 and end - start >= 60:
                        start_time = '%d:%d:%d' % (start // 29.97 // 3600, start//29.97%3600//60, start//29.97%60)
                        end_time = '%d:%d:%d' % (end // 29.97 // 3600, end//29.97%3600//60 , end//29.97%60)
                        f.write(str(start) + "," + str(end)+", " + start_time + "," + end_time+"\n")
                    start = self.frame_cnt
                while True: # 일치하지 않는 구간까지 반복
                    print(self.frame_cnt, result, self.highlight_frame_cnt)
                    self.highlight_frame_cnt += 15
                    self.frame_cnt += 15
                    for i in range(15):
                        self.highlight.read()
                        self.source.read()
                        
                    '''
                    cv2.imshow('source', self.source.frame)
                    cv2.imshow('highlight', self.highlight.frame)
                    key = cv2.waitKey(25)
                    if key == 27:
                        cv2.destroyAllWindows()
                    '''
                    if not self.source.ret or not self.highlight.ret: 
                        break
                    result = self.match_template(self.source.make_grayscale(), self.highlight.make_grayscale())
                    if result > self.precision: # 일치한다면
                        end = self.frame_cnt
                        continue
                    else: # 일치하지 않는다면
                        print(self.frame_cnt, result, self.highlight_frame_cnt)
                        end = self.frame_cnt - 15
                        self.highlight_frame_cnt += 30
                        for i in range(30):
                            self.highlight.read()
                        break
                
        #----------------------------------------------------------------------
        if self.source.cap.isOpened():
            self.source.cap.release()

        if self.highlight.cap.isOpened():
            self.highlight.cap.release()

        #cv2.destroyAllWindows()
        print("END")
        return


In [23]:
text_path = "video_data.txt"
precision = 0.9

with open(text_path, "r") as src:
    while(True):
        source_path = src.readline()
        if not source_path : break
        source_path = source_path.split(',')[1].strip()
        highlight_path = source_path + "_HL.mp4"
        source_path += ".mp4"
        print(source_path)
        finder = Highlight_finder(source_path, highlight_path, precision)
        finder.load_video()
        finder.find_highlight()

T1_vs_DWG.mp4


[ WARN:0@13130.527] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@13130.527] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin16 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@13130.527] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@13130.527] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modu

1.0 min 210
2.0 min 210
3.0 min 210
4.0 min 210
5.0 min 210
10055 0.9862498641014099 210
10070 0.9789513945579529 225
10085 0.9826526045799255 240
10100 0.9850991368293762 255
10115 0.9804286360740662 270
10130 0.9693484902381897 285
10145 0.9777655005455017 300
10160 0.9910120964050293 315
10175 0.9818304777145386 330
10190 0.9576523900032043 345
10205 0.4569174647331238 360
6.0 min 390
7.0 min 390
8.0 min 390
9.0 min 390
10.0 min 390
11.0 min 390
12.0 min 390
13.0 min 390
14.0 min 390
15.0 min 390
16.0 min 390
17.0 min 390
18.0 min 390
19.0 min 390
20.0 min 390
21.0 min 390
22.0 min 390
23.0 min 390
24.0 min 390
25.0 min 390
26.0 min 390
27.0 min 390
28.0 min 390
29.0 min 390
30.0 min 390
31.0 min 390
32.0 min 390
57815 0.9647188782691956 390
57830 0.9998726844787598 405
57845 0.7269710302352905 420
57920 0.9185485243797302 450
57935 0.9999358654022217 465
57950 0.9999389052391052 480
57965 0.8944978713989258 495
58005 0.9513453841209412 525
58020 0.9597196578979492 540
58035 0.96599

102650 0.8969630002975464 4545
102655 0.9063949584960938 4575
102670 0.9128800630569458 4590
102685 0.9079172611236572 4605
102700 0.9084506034851074 4620
102715 0.8978300094604492 4635
102720 0.9016997218132019 4665
102735 0.9070955514907837 4680
102750 0.9011342525482178 4695
102765 0.897777259349823 4710
102880 0.9102559685707092 4740
102895 0.9210171103477478 4755
102910 0.8917648792266846 4770
103040 0.9008331894874573 4800
103055 0.8954209685325623 4815
103060 0.9007818102836609 4845
103075 0.9024289846420288 4860
103090 0.8984766602516174 4875
103095 0.9008488655090332 4905
103110 0.9167442917823792 4920
103125 0.9108671545982361 4935
103140 0.8971714973449707 4950
103200 0.9011178612709045 4980
103215 0.8886186480522156 4995
103220 0.9231430888175964 5025
103235 0.9098160266876221 5040
103250 0.8921208381652832 5055
103345 0.90168696641922 5085
103360 0.9050527215003967 5100
103375 0.8932072520256042 5115
103385 0.9009901285171509 5145
103400 0.9191418290138245 5160
103415 0.92

109370 0.901850163936615 8910
109385 0.8730180263519287 8925
109410 0.9119558334350586 8955
109425 0.9096294045448303 8970
109440 0.906203031539917 8985
109455 0.9029564261436462 9000
109470 0.9120363593101501 9015
109485 0.9137331247329712 9030
109500 0.8930970430374146 9045
109520 0.9070299863815308 9075
109535 0.9069781303405762 9090
109550 0.9080365896224976 9105
109565 0.9232466816902161 9120
109580 0.9232381582260132 9135
109595 0.9023817181587219 9150
109610 0.9089594483375549 9165
109625 0.917359471321106 9180
109640 0.8911269903182983 9195
109650 0.901007890701294 9225
109665 0.9023278951644897 9240
109680 0.9022157788276672 9255
109695 0.9032654762268066 9270
109710 0.9284018278121948 9285
109725 0.9334195256233215 9300
109740 0.9240068197250366 9315
109755 0.8924779891967773 9330
61.0 min 9360
62.0 min 9360
112720 0.9007208347320557 9360
112735 0.8707371354103088 9375
112775 0.9005759954452515 9405
112790 0.9218651652336121 9420
112805 0.9366480112075806 9435
112820 0.929830

123810 0.9235442876815796 13080
123825 0.9275780916213989 13095
123840 0.9290069341659546 13110
123855 0.934410810470581 13125
123870 0.9276972413063049 13140
123885 0.9147264361381531 13155
123900 0.9128950834274292 13170
123915 0.918757438659668 13185
123930 0.9281022548675537 13200
123945 0.937347412109375 13215
123960 0.9363773465156555 13230
123975 0.9439166784286499 13245
123990 0.9319159984588623 13260
124005 0.916128933429718 13275
124020 0.9176167845726013 13290
124035 0.8918083906173706 13305
69.0 min 13335
70.0 min 13335
127475 0.9006564617156982 13335
127490 0.9023861289024353 13350
127505 0.9046323299407959 13365
127520 0.8778350949287415 13380
127530 0.9022316336631775 13410
127545 0.9042322039604187 13425
127560 0.900910496711731 13440
127575 0.9044099450111389 13455
127590 0.9004808664321899 13470
127605 0.9049129486083984 13485
127620 0.8960925936698914 13500
127630 0.904060959815979 13530
127645 0.9136728048324585 13545
127660 0.902470052242279 13560
127675 0.90661948

In [24]:
test_url = "https://m.sports.naver.com/video/970290"
dl.download(test_url, "naver_sports_test")

[generic] 970290: Requesting header


[generic] 970290: Downloading webpage
[generic] 970290: Extracting information


ERROR: Unsupported URL: https://m.sports.naver.com/video/970290


DownloadError: ERROR: Unsupported URL: https://m.sports.naver.com/video/970290